In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from math import sqrt, pi

In [ ]:
file_path = 'fp-historical-wildfire-data-2006-2021.xlsx'
output_file_path = 'updated_file.csv'

df = pd.read_excel(file_path)

df['fire_start_date'] = pd.to_datetime(df['fire_start_date'], errors='coerce')
df['ex_fs_date'] = pd.to_datetime(df['ex_fs_date'], errors='coerce')

start_date = datetime(2006, 1, 1)
end_date = datetime(2021, 12, 31)
monthly_periods = pd.date_range(start=start_date, end=end_date, freq='MS')

for month in monthly_periods:
    column_name = month.strftime('%Y.%b')
    df[column_name] = 0

for index, row in df.iterrows():
    if pd.notna(row['fire_start_date']) and pd.notna(row['ex_fs_date']) and \
       2006 <= row['fire_start_date'].year <= 2021 and \
       2006 <= row['ex_fs_date'].year <= 2021:
        
        start_year_month = datetime(row['fire_start_date'].year, row['fire_start_date'].month, 1)
        end_year_month = datetime(row['ex_fs_date'].year, row['ex_fs_date'].month, 1)
        
        if start_year_month < start_date:
            start_year_month = start_date
        if end_year_month > end_date:
            end_year_month = end_date
        
        for month in monthly_periods:
            if start_year_month <= month <= end_year_month:
                column_name = month.strftime('%Y.%b')
                df.at[index, column_name] = 1

df.to_csv(output_file_path, index=False)


In [ ]:
alberta_gas_path = 'alberta_naturalGas_prod.csv'
updated_wildfire_path = 'updated_file.csv'

gas_df = pd.read_csv(alberta_gas_path)
for col in ['A', 'B', 'C', 'D', 'E']:
    gas_df[col] = 0


wildfire_df = pd.read_csv(updated_wildfire_path)


def map_year_month_to_first(date_str):
    year, month_abbr = date_str.split('.')
    month_number = datetime.strptime(month_abbr, '%b').month
    return f"{year}-{month_number:02d}-01"


for index, wildfire_row in wildfire_df.iterrows():

    size_class = wildfire_row['size_class']
    if size_class in ['A', 'B', 'C', 'D', 'E']:
        for year_month_col in [col for col in wildfire_df.columns if '.' in col]:
            if wildfire_row[year_month_col] == 1:
                gas_date = map_year_month_to_first(year_month_col)
                gas_df.loc[gas_df['Date'] == gas_date, size_class] += 1


gas_df.to_csv('updated_alberta_naturalGas_prod.csv', index=False)


In [ ]:
updated_gas_path = 'updated_alberta_naturalGas_prod.csv'
gas_df = pd.read_csv(updated_gas_path)
gas_df['prod_%_diff'] = gas_df['Value'].pct_change() * 100
gas_df['prod_%_diff'] = gas_df['prod_%_diff'].fillna(0)

gas_df.to_csv('updated_alberta_naturalGas_prod_with_diff.csv', index=False)


In [ ]:
df = pd.read_csv('ST50A.csv')


df['Location Latitude'] = df['Location Latitude'].replace(r'^\s*$', np.nan, regex=True)
df['Location Longitude'] = df['Location Longitude'].replace(r'^\s*$', np.nan, regex=True)

df = df.dropna(how='all')
df_cleaned = df.dropna(subset=['Location Latitude', 'Location Longitude'])
df_cleaned.to_csv('ST50A_cleaned.csv', index=False)


In [ ]:
updated_df = pd.read_csv('updated_file.csv')
st50a_df = pd.read_csv('ST50A_cleaned.csv')

def calc_distance(lat1, lon1, lat2, lon2):
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = 6371 * 1000 * c
    return distance

start_date = pd.to_datetime('2007-02-01')
end_date = pd.to_datetime('2021-10-31')
month_range = pd.date_range(start_date, end_date, freq='MS')

for month in month_range:
    st50a_df[month.strftime('%Y-%m')] = 0

for index, st50a_row in st50a_df.iterrows():
    for _, fire_row in updated_df.iterrows():
        fire_radius = sqrt((fire_row['current_size'] * 10000) / pi) 
        distance = calc_distance(st50a_row['Location Latitude'], st50a_row['Location Longitude'],
                                 fire_row['fire_location_latitude'], fire_row['fire_location_longitude'])
        if pd.notnull(fire_row['fire_start_date']) and distance <= fire_radius:
            fire_start_date = pd.to_datetime(fire_row['fire_start_date']).strftime('%Y-%m')
            if fire_start_date in st50a_df.columns:
                st50a_df.at[index, fire_start_date] = 1

st50a_df.to_csv('ST50A_updated_with_fire_info.csv', index=False)


In [ ]:
updated_df = pd.read_csv('updated_file.csv')
st50a_df = pd.read_csv('monthly_averages.csv')


def calc_distance(lat1, lon1, lat2, lon2):

    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = 6371 * 1000 * c
    return distance


start_date = pd.to_datetime('2007-02-01')
end_date = pd.to_datetime('2021-10-31')
month_range = pd.date_range(start_date, end_date, freq='MS')

for month in month_range:
    st50a_df[month.strftime('%Y-%m')] = 0


for index, st50a_row in st50a_df.iterrows():
    for _, fire_row in updated_df.iterrows():
        fire_radius = sqrt((fire_row['current_size'] * 10000) / pi) + 5000
        distance = calc_distance(st50a_row['Latitude'], st50a_row['Longitude'],
                                 fire_row['fire_location_latitude'], fire_row['fire_location_longitude'])

        if pd.notnull(fire_row['fire_start_date']) and distance <= fire_radius:
            fire_start_date = pd.to_datetime(fire_row['fire_start_date']).strftime('%Y-%m')
            if fire_start_date in st50a_df.columns:
                st50a_df.at[index, fire_start_date] = 1


        if pd.notnull(fire_row['fire_start_date']) and distance <= fire_radius:

            fire_start_date = pd.to_datetime(fire_row['fire_start_date']).strftime('%Y-%m')
            if fire_start_date in st50a_df.columns:
                st50a_df.at[index, fire_start_date] = 1



st50a_df.to_csv('pipeline_affected.csv', index=False)


In [ ]:
st50a_df.to_csv('pipeline_affected.csv', index=False)

In [ ]:
file_path = 'ngtl-throughput-and-capacity (1).csv'
df = pd.read_csv(file_path)

monthly_avg_df = df.groupby(['Longitude', 'Latitude', 'Year', 'Month']).agg({
    'Capacity (1000 m3/d)': 'mean',
    'Throughput (1000 m3/d)': 'mean',
    'Throughput (GJ/d)': 'mean'
}).reset_index()

monthly_avg_df.rename(columns={
    'Capacity (1000 m3/d)': 'Avg Capacity (1000 m3/d)',
    'Throughput (1000 m3/d)': 'Avg Throughput (1000 m3/d)',
    'Throughput (GJ/d)': 'Avg Throughput (GJ/d)'
}, inplace=True)

output_file_path = 'monthly_averages.csv'
monthly_avg_df.to_csv(output_file_path, index=False)